In [1]:
import os
os.chdir('..')

import nmrglue as ng
import numpy as np
from cops_analysis import cops_analyze
from cops_prediction import gaussian
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.cluster import SpectralBiclustering

In [2]:
#predict sequential peaks
class int_seq_match():
    def __init__(self, cops_analyzer, cops_mode='HCA', peak_table_dir=None):
        self.cops_mode = cops_mode
        self.cops_analyzer = cops_analyzer
        

        self.load_peak_table(peak_table_dir)
        self.load_1Ds()
        self.list_mode = True

    
        print("done!")
            
        
    def load_peak_table(self, peak_table_dir):
        '''
        DEFINITION
        __________
        Loads a peak table containing labeled peaks. reads chemical shifts and whether or not peak is sequential or internal.

        PARAMETERS
        __________
        peak_table_dir: string
        location of SPARKY peak table

        OUTPUTS
        __________
        None. This fucntion updates the shifts_array instance and the tb_sequential instance with the peaks from the table.
        '''
        if self.cops_mode == 'HCA':
            self.tb = pd.read_fwf(peak_table_dir, infer_nrows=300)
            self.tb = self.tb.rename(columns={'w1':'CA','w2':'HN'})
                
            self.tb['is_sequential']=np.append([False], [len(self.tb['Assignment'][i+1]) > len(self.tb['Assignment'][i]) for i in range(len(self.tb)-1)])
            self.shifts_array = self.tb[['CA', 'HN']].to_numpy(dtype=np.float32)
            
        elif self.cops_mode == 'HNCA':
            self.tb = pd.read_fwf(peak_table_dir, infer_nrows=300)
            self.tb = self.tb.rename(columns={'w1':'CA','w2':'N','w3':'HN'})

            self.tb['is_sequential']=np.append([False], [len(self.tb['Assignment'][i+1]) > len(self.tb['Assignment'][i]) for i in range(len(self.tb)-1)])
            self.shifts_array = self.tb[['CA', 'N', 'HN']].to_numpy(dtype=np.float32)
            self.shifts_array[:,[0,1]]=self.shifts_array[:,[1,0]]
        
        #self.shifts_array = self.shifts_array[self.tb['is_sequential']]
        self.tb_sequential = self.tb[self.tb['is_sequential']]
        self.tb_internal = self.tb[~self.tb['is_sequential']]
    

    def load_1Ds(self, C_center=None):
        '''
        DEFINITION
        __________
        Loads 1D 13C slices of peaks in the peak table, centered at C_center (ppm).  
        '''
        ca = self.cops_analyzer
        #extract 1Ds from absolute center
        temp_shifts = np.copy(self.shifts_array)
        if C_center != None:
            if self.cops_mode == 'HNCA':
                temp_shifts[:,1] = C_center
            elif self.cops_mode == 'HCA':
                temp_shifts[:,0] = C_center

        self.shifts_index = np.array([[[ca.cop_unit_convs[k][j](temp_shifts[i][j], "ppm") for k in range(ca.cop_num)] for j in range(3)] for i in range(len(temp_shifts))])
        self.slices = np.array([np.array([ca.extract1D(temp_shifts[j], ca.cop_dats[i], ca.cop_unit_convs[i],sw=90, normalize=True)[1] for i in range(ca.cop_num)]).reshape(-1) for j in range(len(temp_shifts))])
        
        try:
            #this would be faster with a vectorizable nmrglue unit conversion object
            self.slices = np.array([np.array([ca.extract1D(temp_shifts[j], ca.cop_dats[i], ca.cop_unit_convs[i],sw=90, normalize=True)[1] for i in range(ca.cop_num)]).reshape(-1) for j in range(len(temp_shifts))])
            self.slices_wide = np.array([np.array([ca.extract1D(temp_shifts[j], ca.cop_dats[i], ca.cop_unit_convs[i],sw=150, normalize=True)[1] for i in range(ca.cop_num)]).reshape(-1) for j in range(len(temp_shifts))])
        except:
            raise ValueError("Error with loading 1D slices. Check peak list")
            

In [3]:
%%time
A = a.cops_analyzer.cop_dats
indices=[[1,3,3,50,2,4],[15,17,3,50,19,22]]
def slices(index):
    return [A[i][index[0]:index[1],index[2]:index[3], index[4]:index[5]] for i in range(len(A))]
P = map(slices, indices)
print(np.fromiter(P, dtype=np.float32))

NameError: name 'a' is not defined

In [4]:
b = cops_analyze(['./pyruvate_HNCA/HNCA_nocop.ft3','./pyruvate_HNCA/HNCA_cop1.ft3','./pyruvate_HNCA/HNCA_cop3.ft3','./pyruvate_HNCA/HNCA_cop4.ft3','./pyruvate_HNCA/HNCA_cop5.ft3','./pyruvate_HNCA/HNCA_cop6.ft3'], mode='HNCA',pyruvate_on=True)

tb_nopyr=pd.read_fwf('./files/GB1_new.shifts', infer_nrows=300)
tb_nopyr = tb_nopyr.rename(columns={'w1':'CA','w2':'N', 'w3':'HN'})

#realign spectra
tb_nopyr = tb_nopyr.set_index(tb_nopyr['Assignment'])
tb_nopyr['is_sequential']=np.append([False], [len(tb_nopyr['Assignment'][i+1]) > len(tb_nopyr['Assignment'][i]) for i in range(len(tb_nopyr)-1)])
#tb_nopyr.loc[tb_nopyr['is_sequential'],'CA']=tb_nopyr[~tb_nopyr['is_sequential']][['CA']][:-1].to_numpy() #enforce the same CA

shifts_array = tb_nopyr[['CA','N', 'HN']].to_numpy(dtype=np.float32)
shifts_array[:,[0,1]]=shifts_array[:,[1,0]]


a = int_seq_match(b, peak_table_dir = './files/GB1_new.shifts', cops_mode='HNCA')
print(tb_nopyr['Assignment'][40])
print(shifts_array[40])
print(a.shifts_index.shape)

done!
D21Ca-N-HN
[115.789  49.417   7.163]
(109, 3, 5)


In [5]:
import cProfile, pstats, io
from pstats import SortKey

pr = cProfile.Profile()
pr.enable()
b = cops_analyze(['./SHP2_Gradcops/SHP2_Grad1.ucsf','./SHP2_Gradcops/SHP2_Grad3.ucsf','./SHP2_Gradcops/SHP2_Grad5.ucsf','./SHP2_Gradcops/SHP2_Grad6.ucsf'], mode='HNCA',pyruvate_on=False, cop_num=[1,3,5,6])
pr.disable()
s = io.StringIO()
sortby = SortKey.CUMULATIVE
ps = pstats.Stats(pr, stream=s).sort_stats(sortby)
ps.print_stats()
print(s.getvalue())

         26411 function calls in 3.298 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        2    0.000    0.000    3.297    1.649 /home/nmrbox/hwang/miniconda3/envs/nmr/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3333(run_code)
        2    0.000    0.000    3.297    1.649 {built-in method builtins.exec}
        1    0.004    0.004    3.297    3.297 /tmp/ipykernel_16671/1202041506.py:6(<cell line: 6>)
        1    0.000    0.000    3.293    3.293 /home/nmrbox/hwang/Documents/COPS_dev/cops_analysis.py:26(__init__)
        1    0.000    0.000    3.285    3.285 /home/nmrbox/hwang/Documents/COPS_dev/cops_analysis.py:112(init_spectra)
        4    0.000    0.000    3.285    0.821 /home/nmrbox/hwang/Documents/COPS_dev/cops_analysis.py:94(import_spectrum)
        4    0.082    0.020    3.285    0.821 /home/nmrbox/hwang/miniconda3/envs/nmr/lib/python3.9/site-packages/nmrglue/fileio/sparky.py:441(read_3D)
     

In [58]:
import cProfile, pstats, io
from pstats import SortKey

pr = cProfile.Profile()
pr.enable()
int_seq_match(b, peak_table_dir = './files/GB1_new.shifts', cops_mode='HNCA')
pr.disable()
s = io.StringIO()
sortby = SortKey.TOTAL
ps = pstats.Stats(pr, stream=s).sort_stats(sortbt)
ps.print_stats()
print(s.getvalue())

done!


AttributeError: TOTAL